In [26]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
import os
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
os.chdir('/content/gdrive/My Drive/tfg_victor') ## mount Google Drive path where files are saved
!ls

Mounted at /content/gdrive
fact_check2.csv		      fact_check.csv	      missinfo-claims.csv
fact_check3.csv		      fact_check_Med1060.csv  model_bool_distil-bert.m
fact_check_bool_balanced.csv  fact_check_Mediano.csv  tfg_victor_res
fact_check_bool.csv	      fact_check_number.csv


In [28]:
df = pd.read_csv('fact_check_Med1060.csv', sep=",")

In [29]:
import pandas as pd

In [30]:
sum = 0
for element in df.veracity.value_counts(): sum = sum + element
for i in range(len(df.veracity.value_counts().index)): print(str(df.veracity.value_counts().index[i]) + " -> " + str(round(100*df.veracity.value_counts()[i]/sum, 1)))

FALSE -> 41.1
BLATANT-LIE -> 39.6
MOSTLY-FALSE -> 8.3
TRUE -> 5.6
MOSTLY-TRUE -> 4.6
NO-EVIDENCE -> 0.7


In [31]:
df
df.veracity.value_counts()

FALSE           365
BLATANT-LIE     351
MOSTLY-FALSE     74
TRUE             50
MOSTLY-TRUE      41
NO-EVIDENCE       6
Name: veracity, dtype: int64

In [32]:
df["veracity"][df["veracity"]=="TRUE"] = 5
df["veracity"][df["veracity"]=="MOSTLY-TRUE"] = 4
df["veracity"][df["veracity"]=="MOSTLY-FALSE"] = 3
df["veracity"][df["veracity"]=="FALSE"] = 2
df["veracity"][df["veracity"]=="BLATANT-LIE"] = 1
df["veracity"][df["veracity"]=="NO-EVIDENCE"] = 0

In [33]:
df.to_csv('fact_check_number.csv', header=True, index=False)
df = pd.read_csv('fact_check_number.csv', sep=',')
df

,veracity,claims
0,2,"The man who killed the Uvalde, Texas, school s..."
1,1,The government gives you your credit score.
2,1,Bill Gates planned a smallpox-type outbreak.
3,1,allegedly shows pallets of boxes containing ba...
4,1,"Active shooter drill at high school in Uvalde,..."
...,...,...
882,2,Recently released Jan. 6 footage shows “these ...
883,2,COVID-19 tests that are labeled “positive cont...
884,2,President Joe Biden “has the lowest approval r...
885,2,Face masks increase the risks of respiratory d...


# Creamos o conxunto de datos booleano

In [34]:
import pandas as pd
#Cargamos o drive
import os
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
os.chdir('/content/gdrive/My Drive/tfg_victor') ## mount Google Drive path where files are saved
!ls

Mounted at /content/gdrive
fact_check2.csv		      fact_check.csv	      missinfo-claims.csv
fact_check3.csv		      fact_check_Med1060.csv  model_bool_distil-bert.m
fact_check_bool_balanced.csv  fact_check_Mediano.csv  tfg_victor_res
fact_check_bool.csv	      fact_check_number.csv


In [35]:
df = pd.read_csv('fact_check_Med1060.csv', sep=",")
df.head()

,veracity,claims
0,FALSE,"The man who killed the Uvalde, Texas, school s..."
1,BLATANT-LIE,The government gives you your credit score.
2,BLATANT-LIE,Bill Gates planned a smallpox-type outbreak.
3,BLATANT-LIE,allegedly shows pallets of boxes containing ba...
4,BLATANT-LIE,"Active shooter drill at high school in Uvalde,..."


In [36]:
df.veracity.value_counts()

FALSE           365
BLATANT-LIE     351
MOSTLY-FALSE     74
TRUE             50
MOSTLY-TRUE      41
NO-EVIDENCE       6
Name: veracity, dtype: int64

In [37]:
indices = df[df["veracity"] == "NO-EVIDENCE"].index
df.drop(indices, inplace=True)

In [38]:
#Cambiamos a calsificación binaria
df["veracity"][df["veracity"]=="TRUE"] = 1
df["veracity"][df["veracity"]=="MOSTLY-TRUE"] = 1
df["veracity"][df["veracity"]=="MOSTLY-FALSE"] = 0
df["veracity"][df["veracity"]=="FALSE"] = 0
df["veracity"][df["veracity"]=="BLATANT-LIE"] = 0

In [39]:
sum = 0
for element in df.veracity.value_counts(): sum = sum + element
for i in range(len(df.veracity.value_counts().index)): print(str(df.veracity.value_counts().index[i]) + " -> " + str(round(100*df.veracity.value_counts()[i]/sum, 1)))

0 -> 89.7
1 -> 10.3


In [40]:
df.to_csv('fact_check_bool.csv', header=True, index=False)
df = pd.read_csv('fact_check_bool.csv', sep=',')
df

,veracity,claims
0,0,"The man who killed the Uvalde, Texas, school s..."
1,0,The government gives you your credit score.
2,0,Bill Gates planned a smallpox-type outbreak.
3,0,allegedly shows pallets of boxes containing ba...
4,0,"Active shooter drill at high school in Uvalde,..."
...,...,...
876,0,Recently released Jan. 6 footage shows “these ...
877,0,COVID-19 tests that are labeled “positive cont...
878,0,President Joe Biden “has the lowest approval r...
879,0,Face masks increase the risks of respiratory d...


# Bibliotecas e funcións para os modelos.

In [41]:
!pip install transformers datasets
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer
from datasets import load_dataset, Value, Features, ClassLabel, load_metric

def tokenize_function(batch):
    return tokenizer(batch["claims"], truncation=True)

'''
Function that evaluates the performance of our classification model
'''
def compute_metrics(eval_preds):
    metric1 = load_metric("accuracy")
    metric2 = load_metric("f1")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric1.compute(predictions=predictions, references=labels)["accuracy"]
    f1c = metric2.compute(predictions=predictions, references=labels, average=None)["f1"]
    return {"accuracy": accuracy, "f1c": f1c}


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Adestramos o modelo coa granularidade dispoible

##bert-base-uncased

In [42]:
data = 'fact_check_number.csv' #fact_check_number.csv bert-base-uncased
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") # distilbert-base-uncased-finetuned-sst-2-english

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/ma

## Feature mapping and label setting

In [43]:
features = Features({'claims': Value('string'),
                     'veracity': ClassLabel(names=['0','1','2','3','4','5'])})
dataset_raw = load_dataset('csv', data_files=data, delimiter=',', features=features)["train"]
print(dataset_raw)
dataset = dataset_raw.shuffle(seed=42).train_test_split(test_size=0.15)

dataset = dataset.map(lambda examples: {'labels': examples['veracity']}, remove_columns=['veracity'])

Using custom data configuration default-16c286fcfb45a74c


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-16c286fcfb45a74c/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['claims', 'veracity'],
    num_rows: 887
})


  0%|          | 0/753 [00:00<?, ?ex/s]

  0%|          | 0/134 [00:00<?, ?ex/s]

In [44]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [45]:
model_number = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)
training_args = TrainingArguments(
    output_dir="./tfg_victor_res", 
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    # per_device_train_batch_size=16,
    # per_device_eval_batch_size=16,
    num_train_epochs=5,
    # weight_decay=0.01,
)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_hea

In [46]:
import time
inicio = time.time()
trainer_number = Trainer(
    model_number,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
trainer_number.evaluate()
trainer_number.train()
fin = time.time()
print(fin - inicio)

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: claims. If claims are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 134
  Batch size = 8


Trainer is attempting to log a value of "[0.         0.         0.11594203 0.16666667 0.         0.        ]" of type <class 'numpy.ndarray'> for key "eval/f1c" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: claims. If claims are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 753
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & 

Epoch,Training Loss,Validation Loss,Accuracy,F1c
1,No log,1.289142,0.477612,[0. 0.38235294 0.60714286 0. 0. 0. ]
2,No log,1.258713,0.477612,[0. 0.58064516 0.5323741 0. 0. 0. ]
3,No log,1.315916,0.462687,[0. 0.59047619 0.48780488 0.1 0. 0. ]
4,No log,1.394725,0.432836,[0. 0.57657658 0.41818182 0.22222222 0. 0. ]
5,No log,1.419451,0.447761,[0. 0.56862745 0.47863248 0.20689655 0. 0. ]


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: claims. If claims are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 134
  Batch size = 8
Trainer is attempting to log a value of "[0.         0.38235294 0.60714286 0.         0.         0.        ]" of type <class 'numpy.ndarray'> for key "eval/f1c" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: claims. If claims are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 134
  Batch size = 8
Trainer is attempting to log a value of "[0.         0.58064516 0.53

55.53457069396973


## Distilbert

In [47]:
data = 'fact_check_number.csv' #fact_check_number.csv bert-base-uncased
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") # distilbert-base-uncased-finetuned-sst-2-english


features = Features({'claims': Value('string'),
                     'veracity': ClassLabel(names=['TRUE', 'MOSTLY-TRUE', 'MOSTLY-FALSE', 'FALSE', 'BLATANT-LIE', 'NO-EVIDENCE'])})
dataset_raw = load_dataset('csv', data_files=data, delimiter=',', features=features)["train"]
print(dataset_raw)
dataset = dataset_raw.shuffle(seed=42).train_test_split(test_size=0.15)

dataset = dataset.map(lambda examples: {'labels': examples['veracity']}, remove_columns=['veracity'])

tokenized_dataset = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

model_number_1 = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=6)
training_args = TrainingArguments(
    output_dir="./tfg_victor_res", 
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    # per_device_train_batch_size=16,
    # per_device_eval_batch_size=16,
    num_train_epochs=5,
    # weight_decay=0.01,
)

https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpdxm6mlyy


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
creating metadata file for /root/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
https://huggingface.co/distilbert-base-uncased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpjcngscnc


Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
creating metadata file for /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.0

...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a
creating metadata file for /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a
loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weigh

In [48]:
import time
inicio = time.time()
trainer_number = Trainer(
    model_number,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
trainer_number.evaluate()
trainer_number.train()
fin = time.time()
print(fin - inicio)

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: claims. If claims are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 134
  Batch size = 8


Trainer is attempting to log a value of "[0.         0.56862745 0.47863248 0.20689655 0.         0.        ]" of type <class 'numpy.ndarray'> for key "eval/f1c" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: claims. If claims are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 753
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & 

Epoch,Training Loss,Validation Loss,Accuracy,F1c
1,No log,1.771743,0.500000,[0. 0.60784314 0.51724138 0.33333333 0.22222222 0.14285714]
2,No log,2.119390,0.425373,[0. 0.57731959 0.46017699 0.17142857 0. 0. ]
3,No log,2.331369,0.500000,[0. 0.6 0.49541284 0.27272727 0.5 0.16666667]
4,No log,2.524776,0.447761,[0. 0.55319149 0.47863248 0.21428571 0.33333333 0.14285714]
5,No log,2.535070,0.455224,[0. 0.55102041 0.47863248 0.24 0.33333333 0.15384615]


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: claims. If claims are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 134
  Batch size = 8
Trainer is attempting to log a value of "[0.         0.60784314 0.51724138 0.33333333 0.22222222 0.14285714]" of type <class 'numpy.ndarray'> for key "eval/f1c" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: claims. If claims are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 134
  Batch size = 8
Trainer is attempting to log a value of "[0.         0.57731959 0.46

58.489291191101074


In [49]:
data = 'fact_check_number.csv' #fact_check_number.csv bert-base-uncased
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") 

#Features e dataset
features = Features({'claims': Value('string'),
                     'veracity': ClassLabel(names=['0','1'])})
dataset_raw = load_dataset('csv', data_files=data, delimiter=',', features=features)['train']
#Poño a liña de separar pq no caso anterior dinos que hay un problema ao ler claim e veracity
dataset = dataset_raw.map(lambda examples: {'labels': examples['veracity']}, remove_columns=['veracity'])

#Tokenizamos o dataset e prepramos o dato_collector
tokenized_dataset = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#Intentamos co modelo gardado pero parece que o estmaos recuperadno mal
#model_path = "model_bool_distil-bert.m"
#model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)
#trainer_aux = Trainer(model)
#raw_pred,_,_ = trainer_aux.predict(tokenized_dataset)
#Empregamos o traine anterior que ten o modelo xa cargado
raw_pred,_,_ = trainer_number.predict(tokenized_dataset)
print(raw_pred)

# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.19.4",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-f75a8807079c88b7/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/887 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: claims. If claims are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 887
  Batch size = 8


[[-2.0119877  -0.9704074   5.7305093  -1.3149129  -1.3901097  -0.69663894]
 [-1.120535    5.6666555  -0.5452424  -1.5398616  -0.7092149  -1.9584169 ]
 [-1.266115    5.8183293  -1.4032581  -1.2236369  -0.4260281  -1.8426163 ]
 ...
 [-2.0010223  -1.8188376   5.917912   -1.275967   -1.3498111  -0.14609076]
 [-2.1555183  -0.48880276  5.486661   -0.6674271  -1.6401368  -1.784623  ]
 [-0.31070647 -2.6773753  -0.3943383  -0.8990167   0.23744656  3.797888  ]]


In [50]:
y_pred

array([2, 1, 1, 1, 1, 2, 3, 1, 5, 1, 2, 1, 3, 5, 2, 2, 3, 1, 4, 4, 2, 1,
       4, 1, 1, 1, 3, 2, 5, 1, 2, 1, 1, 2, 2, 2, 3, 5, 1, 4, 2, 1, 3, 2,
       2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 2, 1, 1, 3, 2, 2, 1, 2, 1,
       3, 2, 2, 5, 1, 1, 1, 2, 5, 2, 1, 1, 1, 1, 4, 4, 2, 1, 1, 1, 1, 1,
       3, 1, 1, 1, 1, 4, 2, 1, 2, 2, 2, 1, 3, 1, 2, 1, 5, 2, 1, 2, 1, 2,
       2, 2, 1, 1, 2, 2, 1, 2, 1, 4, 2, 4, 1, 3, 1, 1, 2, 4, 1, 3, 1, 2,
       2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 3, 1, 1, 2, 2, 2, 2, 1, 2, 2, 5, 1,
       1, 2, 4, 3, 1, 2, 2, 2, 2, 3, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2,
       1, 3, 2, 3, 2, 3, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 5, 4, 1,
       2, 2, 2, 2, 4, 2, 2, 1, 2, 2, 4, 1, 2, 1, 2, 2, 2, 1, 3, 1, 5, 2,
       2, 1, 1, 1, 3, 2, 1, 3, 1, 2, 2, 2, 1, 3, 1, 2, 2, 2, 2, 2, 2, 1,
       2, 3, 3, 1, 1, 3, 2, 5, 2, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 2, 4, 1,
       2, 3, 3, 5, 1, 2, 2, 3, 1, 4, 1, 2, 5, 1, 5, 2, 1, 2, 2, 1, 2, 1,
       1, 2, 1, 2, 2, 1, 1, 2, 2, 5, 1, 1, 2, 2, 4,

# Adestramos o modelo cun conxunto binario desbalanceado.

In [51]:
#### Computamos o desbalanceo
df = pd.read_csv('fact_check_bool.csv')
cw = {k:(max(dict(df["veracity"].value_counts()).values())/v) for k,v in dict(df["veracity"].value_counts()).items()}

counts = [dict(df["veracity"].value_counts())[0], dict(df["veracity"].value_counts())[1]]
n_samples = counts[0] + counts[1]
n_classes = len(counts)  # wj=n_samples / (n_classes * n_samplesj)
class_weights = [n_samples / (n_classes * n_samplesj) for n_samplesj in counts]
cw=class_weights

In [ ]:
cw

In [52]:
######## PARA ADESTRAR UN MODELO DESBALANCEADO TEMOS QUE SOBREESCRIBIR A CLASE TRAINER E CREAR UN CUSTOM TRAINER QUE TEÑA EN CONTA O DESBALANCEO (PROPORCIÓN ENTRE AS CLASES)
import torch
import torch.nn as nn

class CustomTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get('logits')
        # compute custom loss
        weights = torch.tensor(cw, dtype=torch.float32).to('cuda:0')
        loss_fct = nn.CrossEntropyLoss(weight=weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

## bert-base-uncased

In [53]:
data = 'fact_check_bool.csv' #fact_check_number.csv bert-base-uncased   roberta-large
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") # distilbert-base-uncased-finetuned-sst-2-english

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/ma

In [58]:
#Features e dataset
features = Features({'claims': Value('string'),
                     'veracity': ClassLabel(names=['0','1'])})
dataset_raw = load_dataset('csv', data_files=data, delimiter=',', features=features)["train"]
print(dataset_raw)
dataset = dataset_raw.shuffle(seed=42).train_test_split(test_size=0.15)

dataset = dataset.map(lambda examples: {'labels': examples['veracity']}, remove_columns=['veracity'])

#Tokenizamos o dataset e prepramos o dato_collector
tokenized_dataset = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)



Using custom data configuration default-668fcb7df8b2b93f
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-668fcb7df8b2b93f/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/default-668fcb7df8b2b93f/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-c9e2e533e9335fb6.arrow


Dataset({
    features: ['claims', 'veracity'],
    num_rows: 881
})


  0%|          | 0/748 [00:00<?, ?ex/s]

  0%|          | 0/133 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [59]:
# Configuramos o modelo
model_bool = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to('cuda:0')
training_args = TrainingArguments(
    output_dir="./tfg_victor_res", 
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    # per_device_train_batch_size=16,
    # per_device_eval_batch_size=16,
    num_train_epochs=5,
    # weight_decay=0.01,
)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/re

In [60]:
#Executamos o adestramento
trainer_bool = CustomTrainer(
    model_bool,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
#trainer_bool.evaluate()
#trainer_bool.train()

In [61]:
import time
inicio = time.time()
eval = trainer_bool.evaluate()
train = trainer_bool.train()
fin = time.time()
print(fin - inicio)

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: claims. If claims are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 133
  Batch size = 8


Trainer is attempting to log a value of "[0.01694915 0.21621622]" of type <class 'numpy.ndarray'> for key "eval/f1c" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: claims. If claims are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 748
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation st

Epoch,Training Loss,Validation Loss,Accuracy,F1c
1,No log,0.724508,0.879699,[0.936 0. ]
2,No log,1.468787,0.872180,[0.93172691 0. ]
3,No log,1.993959,0.864662,[0.92741935 0. ]
4,No log,1.285622,0.812030,[0.89451477 0.13793103]
5,No log,1.747551,0.827068,[0.90376569 0.14814815]


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: claims. If claims are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 133
  Batch size = 8
Trainer is attempting to log a value of "[0.936 0.   ]" of type <class 'numpy.ndarray'> for key "eval/f1c" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: claims. If claims are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 133
  Batch size = 8
Trainer is attempting to log a value of "[0.93172691 0.        ]" of type <class 'numpy.ndarray'> for key "eval/f1c" as a 

52.67212629318237


In [62]:
data = 'fact_check_bool.csv' #fact_check_number.csv bert-base-uncased
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") 

#Features e dataset
features = Features({'claims': Value('string'),
                     'veracity': ClassLabel(names=['0','1'])})
dataset_raw = load_dataset('csv', data_files=data, delimiter=',', features=features)['train']
#Poño a liña de separar pq no caso anterior dinos que hay un problema ao ler claim e veracity
dataset = dataset_raw.map(lambda examples: {'labels': examples['veracity']}, remove_columns=['veracity'])

#Tokenizamos o dataset e prepramos o dato_collector
tokenized_dataset = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#Intentamos co modelo gardado pero parece que o estmaos recuperadno mal
#model_path = "model_bool_distil-bert.m"
#model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)
#trainer_aux = Trainer(model)
#raw_pred,_,_ = trainer_aux.predict(tokenized_dataset)
#Empregamos o traine anterior que ten o modelo xa cargado
raw_pred,_,_ = trainer_bool.predict(tokenized_dataset)
print(raw_pred)

# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)
y_pred

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.19.4",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/881 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: claims. If claims are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 881
  Batch size = 8


[[ 2.7457848 -3.032604 ]
 [ 2.6306288 -2.959827 ]
 [ 3.01448   -3.185243 ]
 ...
 [ 2.515514  -2.698255 ]
 [ 3.0516143 -3.1939409]
 [-1.7194033  1.6485212]]


array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,

## distilbert-base-uncased

In [63]:
data = 'fact_check_bool.csv' #fact_check_number.csv bert-base-uncased
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") 

#Features e dataset
features = Features({'claims': Value('string'),
                     'veracity': ClassLabel(names=['0','1'])})
dataset_raw = load_dataset('csv', data_files=data, delimiter=',', features=features)["train"]
print(dataset_raw)
dataset = dataset_raw.shuffle(seed=42).train_test_split(test_size=0.15)

dataset = dataset.map(lambda examples: {'labels': examples['veracity']}, remove_columns=['veracity'])

#Tokenizamos o dataset e prepramos o dato_collector
tokenized_dataset = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


# Configuramos o modelo
model_bool = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
training_args = TrainingArguments(
    output_dir="./tfg_victor_res", 
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    # per_device_train_batch_size=16,
    # per_device_eval_batch_size=16,
    num_train_epochs=5,
    # weight_decay=0.01,
)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.19.4",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10

  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/default-668fcb7df8b2b93f/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-c9e2e533e9335fb6.arrow
Loading cached split indices for dataset at /root/.cache/huggingface/datasets/csv/default-668fcb7df8b2b93f/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-4f846d8ec0416f7d.arrow and /root/.cache/huggingface/datasets/csv/default-668fcb7df8b2b93f/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-29e84d5a25eb2a09.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-668fcb7df8b2b93f/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-f154bd9df5b8cb73.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-668fcb7df8b2b93f/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-ee24a2069674cf95.arrow


Dataset({
    features: ['claims', 'veracity'],
    num_rows: 881
})


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.19.4",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9c169103d7e5a

In [64]:
import time
inicio = time.time()
trainer_bool = CustomTrainer(
    model_bool,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
eval = trainer_bool.evaluate()
train = trainer_bool.train()
fin = time.time()
print(fin - inicio)

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: claims. If claims are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 133
  Batch size = 8


Trainer is attempting to log a value of "[0.93172691 0.        ]" of type <class 'numpy.ndarray'> for key "eval/f1c" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: claims. If claims are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 748
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Acc

Epoch,Training Loss,Validation Loss,Accuracy,F1c
1,No log,0.724717,0.879699,[0.936 0. ]
2,No log,0.723162,0.872180,[0.93117409 0.10526316]
3,No log,1.142841,0.887218,[0.93877551 0.28571429]
4,No log,1.186714,0.864662,[0.925 0.30769231]
5,No log,1.220290,0.857143,[0.92050209 0.2962963 ]


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: claims. If claims are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 133
  Batch size = 8
Trainer is attempting to log a value of "[0.936 0.   ]" of type <class 'numpy.ndarray'> for key "eval/f1c" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: claims. If claims are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 133
  Batch size = 8
Trainer is attempting to log a value of "[0.93117409 0.10526316]" of type <class 'numpy.ndarray'> 

28.332922220230103


In [65]:
data = 'fact_check_bool.csv' #fact_check_number.csv bert-base-uncased
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") 

#Features e dataset
features = Features({'claims': Value('string'),
                     'veracity': ClassLabel(names=['0','1'])})
dataset_raw = load_dataset('csv', data_files=data, delimiter=',', features=features)['train']
#Poño a liña de separar pq no caso anterior dinos que hay un problema ao ler claim e veracity
dataset = dataset_raw.map(lambda examples: {'labels': examples['veracity']}, remove_columns=['veracity'])

#Tokenizamos o dataset e prepramos o dato_collector
tokenized_dataset = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#Intentamos co modelo gardado pero parece que o estmaos recuperadno mal
#model_path = "model_bool_distil-bert.m"
#model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)
#trainer_aux = Trainer(model)
#raw_pred,_,_ = trainer_aux.predict(tokenized_dataset)
#Empregamos o traine anterior que ten o modelo xa cargado
raw_pred,_,_ = trainer_bool.predict(tokenized_dataset)
print(raw_pred)

# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)
y_pred

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.19.4",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10

  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-668fcb7df8b2b93f/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-db00a7cf71aaefc9.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-668fcb7df8b2b93f/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-541e00b3a583ce56.arrow
The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: claims. If claims are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 881
  Batch size = 8


[[ 1.8982867 -1.9667265]
 [ 2.229551  -2.090162 ]
 [ 2.2698808 -2.274195 ]
 ...
 [ 1.293546  -1.3067374]
 [ 2.6431015 -2.5101995]
 [-1.727813   1.5308313]]


array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,

In [66]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,

In [67]:
tokenized_dataset

Dataset({
    features: ['claims', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 881
})

In [68]:
tokenized_dataset

Dataset({
    features: ['claims', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 881
})

# EXEMPLO FUNCIONAL

In [69]:
df = pd.read_csv('missinfo-claims.csv', sep=',')
df.veracity.value_counts()

0    25
1    25
Name: veracity, dtype: int64

In [70]:
trainer_bool.save_model("model_bool_distil-bert.m")

Saving model checkpoint to model_bool_distil-bert.m
Configuration saved in model_bool_distil-bert.m/config.json
Model weights saved in model_bool_distil-bert.m/pytorch_model.bin
tokenizer config file saved in model_bool_distil-bert.m/tokenizer_config.json
Special tokens file saved in model_bool_distil-bert.m/special_tokens_map.json


In [71]:
#from transformers import AutoModel
#model_bool_distil_trained = AutoModelForSequenceClassification.from_pretrained("model_bool_distil-bert.m")

In [72]:
## Proba de execución do modelo
##VICTOR

data = 'missinfo-claims.csv' #fact_check_number.csv bert-base-uncased
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased") 

#Features e dataset
features = Features({'claims': Value('string'),
                     'veracity': ClassLabel(names=['0','1'])})
dataset_raw = load_dataset('csv', data_files=data, delimiter=',', features=features)['train']
#Poño a liña de separar pq no caso anterior dinos que hay un problema ao ler claim e veracity
dataset = dataset_raw.map(lambda examples: {'labels': examples['veracity']}, remove_columns=['veracity'])

#Tokenizamos o dataset e prepramos o dato_collector
tokenized_dataset = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#Intentamos co modelo gardado pero parece que o estmaos recuperadno mal
#model_path = "model_bool_distil-bert.m"
#model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)
#trainer_aux = Trainer(model)
#raw_pred,_,_ = trainer_aux.predict(tokenized_dataset)
#Empregamos o traine anterior que ten o modelo xa cargado
raw_pred,_,_ = trainer_bool.predict(tokenized_dataset)
print(raw_pred)

# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1) #### Esto devólveche as clases predicidas e coas labels podes computar a accuracy e o f1, cas funcións que tés arriba

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.19.4",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a4a6e98552b25254/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: claims. If claims are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 50
  Batch size = 8


[[ 2.4562306 -2.406081 ]
 [ 2.4559896 -2.3602316]
 [ 2.3104107 -2.178885 ]
 [ 2.4533267 -2.445607 ]
 [ 2.123322  -2.1324706]
 [ 2.5535014 -2.4385884]
 [ 2.2879968 -2.241595 ]
 [ 2.5680518 -2.442685 ]
 [ 2.5115147 -2.4264622]
 [ 2.5909667 -2.506677 ]
 [ 2.4209466 -2.3425453]
 [ 2.5203393 -2.4476333]
 [ 2.398897  -2.3594847]
 [ 2.5223117 -2.4576888]
 [ 2.37175   -2.29376  ]
 [ 2.2465286 -2.2564118]
 [ 2.5435464 -2.4334517]
 [ 2.5161443 -2.4258842]
 [ 2.1918404 -2.1827998]
 [ 2.5418224 -2.429655 ]
 [ 2.2684135 -2.318557 ]
 [ 2.3895807 -2.3621817]
 [ 2.5536242 -2.4630766]
 [ 2.0557609 -2.0538309]
 [ 2.3119936 -2.2328131]
 [ 2.6391323 -2.4858096]
 [ 2.4784155 -2.353577 ]
 [ 2.2656012 -2.2330885]
 [ 2.2277415 -2.2151806]
 [ 2.2947204 -2.2661788]
 [ 2.3410726 -2.3086932]
 [ 2.5687983 -2.4582803]
 [ 2.2026963 -2.1462605]
 [ 2.573239  -2.4764795]
 [ 2.5672238 -2.4722314]
 [ 2.4458022 -2.3465667]
 [ 2.5473514 -2.4570775]
 [ 2.455447  -2.3968205]
 [ 2.5653625 -2.4260502]
 [ 2.6490746 -2.464801 ]


In [73]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [74]:
count = 0
for a in dataset['labels']:
  if a == 0:
    count = count + 1

count

25

In [75]:
metric1 = load_metric("accuracy")
accuracy = metric1.compute(predictions=y_pred, references=dataset['labels'])["accuracy"]
print("Accuracy: " + str(accuracy))

metric2 = load_metric("f1")
f1 = metric2.compute(predictions=y_pred, references=dataset['labels'], average='macro')["f1"]
print("f1: " + str(f1))

metric5 = load_metric("f1")
f1c = metric5.compute(predictions=y_pred, references=dataset['labels'], average=None)["f1"]
print("f1: " + str(f1c))

metric3 = load_metric("recall")
recall = metric3.compute(predictions=y_pred, references=dataset['labels'])["recall"]
print("recall: " + str(recall))

metric4 = load_metric("precision")
precision = metric4.compute(predictions=y_pred, references=dataset['labels'])["precision"]
print("precission: " + str(precision))

Accuracy: 0.5
f1: 0.3333333333333333
f1: [0.66666667 0.        ]


recall: 0.0


precission: 0.0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Entorno de execución

In [76]:
!nvidia-smi

Mon Jun 13 06:44:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    31W /  70W |   7276MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [77]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2199.998
BogoMIPS:            4399.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

In [78]:
!free

              total        used        free      shared  buff/cache   available
Mem:       13298572     6089620      156492       15552     7052460     9387080
Swap:             0           0           0
